The deterministic model is trained based on golden labels(manually labeled), the model takes in one data sample statistics as input and predict the sample label. The input data abstains Context_Specific class. The general accuracy is `93.445`.

## After prepossessing

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Model
from keras.layers import Dense, Embedding, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPool1D, Dropout, concatenate
from keras.preprocessing import text as keras_text, sequence as keras_seq
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import accuracy_score, confusion_matrix
np.random.seed(512)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from keras.layers import LeakyReLU
from keras import initializers
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from keras.models import load_model
from keras.utils import to_categorical
import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold,StratifiedKFold
from keras.models import load_model

In [0]:
# base = 'drive/My Drive/snow/downsteam_model/'
source = './../results/'
data = pd.read_csv(source+'processed_raw_4.csv')
labels = pd.read_csv(source+'train_marginals_4.csv',header=None).values
data.shape, labels.shape

In [0]:
data.head()

,Unnamed: 0,Unnamed: 0.1,y_act,Reason,Attribute_name,Total_val,num_of_dist_val,% nans,max_val,mean,...,sample_5,0,1,2,3,p_dist_val,p_nans,castability,extractability,len_val
0,0,0,3,l,StratificationCategory2,403984,1,0.803648,0.0,0.0,...,NaN,0.071483,0.071483,0.071483,0.785551,0.000002,0.803648,0,0,0.0
1,1,1,3,l,Stratification2,403984,1,0.803648,0.0,0.0,...,NaN,0.071483,0.071483,0.071483,0.785551,0.000002,0.803648,0,0,0.0
2,2,2,3,l,StratificationCategory3,403984,1,0.803648,0.0,0.0,...,NaN,0.071483,0.071483,0.071483,0.785551,0.000002,0.803648,0,0,0.0
3,3,3,3,l,Stratification3,403984,1,0.803648,0.0,0.0,...,NaN,0.071483,0.071483,0.071483,0.785551,0.000002,0.803648,0,0,0.0
4,4,4,2,c,GeoLocation,403984,55,0.006357,0.0,0.0,...,"(37.63864012300047, -120.99999953799971)",0.095930,0.027548,0.848974,0.027548,0.000136,0.006357,1,0,2.0


In [0]:
def abs_limit(x):
    if abs(x) > 10000:
        return 10000*np.sign(x)
    return x

In [0]:
data = data.fillna(0)
data['scaled_mean'] = data['mean'].apply(abs_limit)
data['scaled_min_val'] = data['min_val'].apply(abs_limit)
data['scaled_max_val'] = data['max_val'].apply(abs_limit)
data['scaled_std_dev'] = data['std_dev'].apply(abs_limit)

In [0]:
column_names_to_normalize = ['scaled_max_val', 'scaled_mean', 'scaled_min_val','scaled_std_dev']
x = data[column_names_to_normalize].values
x = np.nan_to_num(x)
x_scaled = StandardScaler().fit_transform(x)
df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = data.index)
data[column_names_to_normalize] = df_temp

In [0]:
data.head()

,Unnamed: 0,Unnamed: 0.1,y_act,Reason,Attribute_name,Total_val,num_of_dist_val,% nans,max_val,mean,...,3,p_dist_val,p_nans,castability,extractability,len_val,scaled_mean,scaled_min_val,scaled_max_val,scaled_std_dev
0,0,0,3,l,StratificationCategory2,403984,1,0.803648,0.0,0.0,...,0.785551,0.000002,0.803648,0,0,0.0,-0.289384,0.032129,-0.41314,-0.351573
1,1,1,3,l,Stratification2,403984,1,0.803648,0.0,0.0,...,0.785551,0.000002,0.803648,0,0,0.0,-0.289384,0.032129,-0.41314,-0.351573
2,2,2,3,l,StratificationCategory3,403984,1,0.803648,0.0,0.0,...,0.785551,0.000002,0.803648,0,0,0.0,-0.289384,0.032129,-0.41314,-0.351573
3,3,3,3,l,Stratification3,403984,1,0.803648,0.0,0.0,...,0.785551,0.000002,0.803648,0,0,0.0,-0.289384,0.032129,-0.41314,-0.351573
4,4,4,2,c,GeoLocation,403984,55,0.006357,0.0,0.0,...,0.027548,0.000136,0.006357,1,0,2.0,-0.289384,0.032129,-0.41314,-0.351573


In [0]:
data.columns

Index([u'Unnamed: 0', u'Unnamed: 0.1', u'y_act', u'Reason', u'Attribute_name',
       u'Total_val', u'num_of_dist_val', u'% nans', u'max_val', u'mean',
       u'min_val', u'std_dev', u'sample_1', u'sample_2', u'sample_3',
       u'sample_4', u'sample_5', u'0', u'1', u'2', u'3', u'p_dist_val',
       u'p_nans', u'castability', u'extractability', u'len_val',
       u'scaled_mean', u'scaled_min_val', u'scaled_max_val',
       u'scaled_std_dev'],
      dtype='object')

In [0]:
data_train, data_test = train_test_split(data, test_size=0.2, random_state=100)
data_train.shape, data_test.shape

((6952, 30), (1739, 30))

In [0]:
x_columns =  ['Attribute_name',
                  'sample_1',
                  'sample_2',
                  'sample_3',
                  'sample_4',
                  'sample_5',
                  'p_nans',
                  'p_dist_val',
                  'scaled_max_val', 
                  'scaled_mean', 
                  'scaled_min_val',
                  'scaled_std_dev',
                  'castability',
                  'extractability',
                  'len_val']

y_columns = ['0','1','2','3']

X_train = data_train[x_columns].values
X_test = data_test[x_columns].values
y_train = data_train[y_columns].values.astype(float)
y_test = data_test[y_columns].values.astype(float)
y_act = data_test['y_act'].values.astype(int)
X_train.shape, X_test.shape, y_train.shape, y_test.shape, y_act.shape

((6952, 15), (1739, 15), (6952, 4), (1739, 4), (1739,))

In [0]:
sample_base = 'sample_'
# samples_train = []
# samples_test = []

i = 1
field_name = sample_base + str(i)
field_idx = x_columns.index(field_name)

samples1_train = X_train[:,field_idx].astype(str)
samples1_test = X_test[:,field_idx].astype(str)

In [0]:
field_name = 'Attribute_name'
atr_Xtrain = X_train[:,x_columns.index(field_name)].astype(str)
atr_Xtest = X_test[:,x_columns.index(field_name)].astype(str)

In [0]:
# define network parameters
max_features = 128
maxlen = 128

In [0]:
def generate_padded_tokens(train_list, test_list, maxlen=maxlen):
    tokenizer = keras_text.Tokenizer(char_level = True)
    tokenizer.fit_on_texts(train_list)
    tokenized_train = tokenizer.texts_to_sequences(train_list)
    pad_train = keras_seq.pad_sequences(tokenized_train, maxlen)

    tokenized_test = tokenizer.texts_to_sequences(test_list)
    pad_test = keras_seq.pad_sequences(tokenized_test, maxlen)
    return tokenizer, pad_train, pad_test

In [0]:
atr_tokenizer, atr_pad_train, atr_pad_test = generate_padded_tokens(atr_Xtrain, atr_Xtest)
sam_tokenizer, sam_pad_train, sam_pad_test = generate_padded_tokens(samples1_train, samples1_test)

In [0]:
X_train_left = X_train[:,6:]
X_test_left = X_test[:,6:]
X_train_left.shape, X_test_left.shape

((6952, 9), (1739, 9))

In [0]:
X_t = atr_pad_train
X_t1 = sam_pad_train
structured_data_train = X_train_left
X_te = atr_pad_test
X_te1 = sam_pad_test
structured_data_test = X_test_left
X_t.shape, X_t1.shape, structured_data_train.shape

((6952, 128), (6952, 128), (6952, 9))

In [0]:
import datetime
print('============================================================')
print('training starts at: \t', datetime.datetime.now())
print('============================================================\n')

('training starts at: \t', datetime.datetime(2019, 3, 7, 22, 10, 29, 922316))



In [0]:
# http://sujitpal.blogspot.com/2018/02/using-snorkel-probabilistic-labels-for.html

def build_model(numfilters, embed_size, num_classes, tokenizer1, tokenizer2, maxlen):
    # name space input
    seq_input = Input(shape=(maxlen,))
    x = Embedding(input_dim = len(tokenizer1.word_counts)+1, output_dim = embed_size)(seq_input)
    x = Dropout(0.25)(x)
    x = Conv1D(numfilters*2**(i), kernel_size = 3, activation = 'relu')(x)
    out_conv = [Dropout(0.5)(GlobalMaxPool1D()(x)), GlobalMaxPool1D()(x)]
    seq1 = concatenate(out_conv, axis = -1)  

    # sample space input
    seq_input2 = Input(shape=(maxlen, ))
    x = Embedding(input_dim = len(tokenizer2.word_counts)+1, output_dim = embed_size)(seq_input2)
    x = Dropout(0.25)(x)
    x = Conv1D(numfilters*2**(i), kernel_size = 3, activation = 'relu')(x)
    out_conv = [Dropout(0.5)(GlobalMaxPool1D()(x)), GlobalMaxPool1D()(x)]
    seq2 = concatenate(out_conv, axis = -1)

    # feature space input
    list_input = Input(shape=(9,))

    layersfin = keras.layers.concatenate([seq1, seq2, list_input])
    x = Dense(500, activation='relu')(layersfin)
    x = Dropout(0.25)(x)
    x = Dense(500, activation='relu')(x)
    x = Dropout(0.1)(x)
    
    preds = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=[seq_input,seq_input2,list_input], outputs=[preds])
    return model

def compile_model(model):
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

def fit_model(model, best_model_file, Xtrain, Ytrain, epochs=20, verbose=1):
    checkpoint = ModelCheckpoint(filepath=best_model_file,
                                 monitor='val_acc', 
                                 verbose=verbose,
                                 save_best_only=True)

    early = EarlyStopping(monitor="val_acc", patience=20)
    callbacks_list = [checkpoint, early] #early            

    history = model.fit(Xtrain, Ytrain, 
                        validation_split=0.1, 
                        epochs=epochs, 
                        batch_size=64,
                        verbose=verbose,
                        callbacks=callbacks_list)
    return history
  
def eval_report(title, Ytest, Ytest_, prob, verbose=True):
    if prob:
        ytest = np.argmax(Ytest, axis=1)
    else:
        ytest = Ytest
    ytest_ = np.argmax(Ytest_, axis=1)
    acc = accuracy_score(ytest, ytest_)
    cm = confusion_matrix(ytest, ytest_)
    if verbose:
      print("\n*** {:s}".format(title.upper()))
      print("accuracy: {:.3f}".format(acc*100))
      print("confusion matrix")
      print(cm)
    return acc

In [0]:
X_t = atr_pad_train
X_t1 = sam_pad_train
structured_data_train = X_train_left
X_te = atr_pad_test
X_te1 = sam_pad_test
structured_data_test = X_test_left
X_t.shape, X_t1.shape, structured_data_train.shape

Xtrain = [X_t, X_t1, structured_data_train]
Xtest = [X_te, X_te1, structured_data_test]
Yptrain = y_train
Yptest = y_test
Yctest = y_act
X_t.shape, X_te.shape, Yptrain.shape, Yptest.shape, Yctest.shape

((6952, 128), (1739, 128), (6952, 4), (1739, 4), (1739,))

In [0]:
nes = [16, 32, 64, 128, 512]
mds = [16, 32, 64, 128]
epoches = [10, 20, 50, 100]

num_classes = Yptrain.shape[1]

scores = list()
with open(output_base+'log.txt', 'w') as fw:
  fw.write('[{:5s} {:5s} {:5s}]  -  {:6s}  -  {:6s}  -  {:6s}'.format('ne', 'md', 'epoch','train','test','real'))
  fw.write('\n')
  for i, ne in enumerate(nes):
    for j, md in enumerate(mds):
      for k, epoch in enumerate(epoches):
        model_p = build_model(ne, md, num_classes, atr_tokenizer, sam_tokenizer, maxlen)
        model_p = compile_model(model_p)

        BEST_MODEL_P = output_base+'best_weights='+\
                        '_ne'+str(ne)+\
                        '_md'+str(md)+\
                        '_ep'+str(epoch)+'.h5'
        fit_model(model_p, BEST_MODEL_P, Xtrain, Yptrain, epoch, verbose=0)
        best_model_p = load_model(BEST_MODEL_P)
        Yptest_ = best_model_p.predict(Xtest)
        score0 = eval_report("test. probabilistic VS pred. probabilistic", Yptest, Yptest_, prob=True, verbose=False)
        score1 = eval_report("test. categorical VS pred. probabilistic", Yctest, Yptest_, prob=False, verbose=False)
        score2 = eval_report("test. categorical VS test. probabilistic", Yctest, Yptest, prob=False, verbose=False)
        scores.append((ne, md, epoch, score0, score1, score2))
        print('[%5d %5d %5d]\t' % (ne, md, epoch), 'Disc. Model Acc.: ', round(score1*100,2))
        fw.write('[{:5d} {:5d} {:5d}]  -    {:.2f}  -    {:.2f}  -    {:.2f}'.format(ne, md, epoch,
                                                                               round(score0*100,2),
                                                                               round(score1*100,2),
                                                                               round(score2*100,2)))
        fw.write('\n')
        fw.flush()

In [0]:
ne=128
md=128
epoch=20
num_classes = Yptrain.shape[1]

model_p = build_model(ne, md, num_classes, atr_tokenizer, sam_tokenizer, maxlen)
model_p = compile_model(model_p)

BEST_MODEL_P = 'best_weights'+str(ne)+str(md)+'.h5'

fit_model(model_p, BEST_MODEL_P, Xtrain, Yptrain, epoch, verbose=0)

best_model_p = load_model(BEST_MODEL_P)

In [0]:
Yptest_ = best_model_p.predict(Xtest)
eval_report("test. probabilistic VS pred. probabilistic", Yptest, Yptest_, prob=True)
eval_report("test. categorical VS pred. probabilistic", Yctest, Yptest_, prob=False)
eval_report("test. categorical VS test. probabilistic", Yctest, Yptest, prob=False)


*** TEST. PROBABILISTIC VS PRED. PROBABILISTIC
accuracy: 87.292
confusion matrix
[[401  89   9   8]
 [ 22 897   0   8]
 [ 12  18  53   1]
 [ 32  14   8 167]]

*** TEST. CATEGORICAL VS PRED. PROBABILISTIC
accuracy: 84.416
confusion matrix
[[340  84   1   6]
 [ 52 921   0  26]
 [ 40   6  64   9]
 [ 35   7   5 143]]

*** TEST. CATEGORICAL VS TEST. PROBABILISTIC
accuracy: 85.509
confusion matrix
[[374  50   4   3]
 [ 84 872  16  27]
 [ 39   2  64  14]
 [ 10   3   0 177]]


0.8550891316848763